In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# 데이터 경로 설정
base_path = 'resized_skindata'  # 경로를 실제 Google Drive에 있는 skindata 폴더 경로로 변경하세요.

# 얼굴 부위 및 피부 상태 설정
face_parts = ['forehead', 'nose', 'cheek']
skin_states = ['dry', 'oily']

# 데이터 불러오기
data = []
labels = []

for part in face_parts:
    for state in skin_states:
        folder_path = os.path.join(base_path, 'train', state, part)
        for filename in os.listdir(folder_path):
            img = cv2.imread(os.path.join(folder_path, filename))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # OpenCV는 BGR 형식이므로 RGB로 변환
            img = cv2.resize(img, (128, 128))  # 이미지 크기 조절

            data.append(img)
            labels.append(f'{state}_{part}')

# 데이터와 레이블을 numpy 배열로 변환
data = np.array(data)
labels = np.array(labels)

# 레이블 인코딩
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
encoded_labels = to_categorical(encoded_labels)

# 데이터 분할 (8:2 비율)
X_train, X_test, y_train, y_test = train_test_split(data, encoded_labels, test_size=0.2, random_state=42)

In [2]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(512, activation='relu'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
15/15 [==============================] - 132s 9s/step - loss: 2.0518 - accuracy: 0.4331 - val_loss: 203.4134 - val_accuracy: 0.2417
Epoch 2/10
15/15 [==============================] - 116s 8s/step - loss: 1.3485 - accuracy: 0.5837 - val_loss: 5355.1045 - val_accuracy: 0.2417
Epoch 3/10
15/15 [==============================] - 108s 7s/step - loss: 0.7712 - accuracy: 0.7113 - val_loss: 8.8794 - val_accuracy: 0.2083
Epoch 4/10
15/15 [==============================] - 137s 9s/step - loss: 0.5792 - accuracy: 0.7845 - val_loss: 5.7056 - val_accuracy: 0.1917
Epoch 5/10
15/15 [==============================] - 118s 8s/step - loss: 0.4564 - accuracy: 0.8389 - val_loss: 4.1249 - val_accuracy: 0.3667
Epoch 6/10
15/15 [==============================] - 125s 8s/step - loss: 0.2953 - accuracy: 0.9059 - val_loss: 6.1194 - val_accuracy: 0.1917
Epoch 7/10
15/15 [==============================] - 117s 8s/step - loss: 0.3486 - accuracy: 0.8703 - val_loss: 3.5041 - val_accuracy: 0.2583
Epoch 8/

In [3]:
# 모델 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

# 테스트 데이터에 대한 예측
predictions = model.predict(X_test)

# 예측 결과 출력
for i in range(5):  # 예측 결과 중에서 처음 5개만 출력
    predicted_class = label_encoder.classes_[np.argmax(predictions[i])]
    actual_class = label_encoder.classes_[np.argmax(y_test[i])]
    print(f'Actual: {actual_class}, Predicted: {predicted_class}')

4/4 [==============================] - 5s 1s/step - loss: 2.0586 - accuracy: 0.5250
Test Loss: 2.0586066246032715, Test Accuracy: 0.5249999761581421
Actual: oily_forehead, Predicted: dry_nose
Actual: oily_nose, Predicted: oily_nose
Actual: oily_cheek, Predicted: oily_cheek
Actual: dry_forehead, Predicted: oily_nose
Actual: dry_nose, Predicted: dry_nose
